In [1]:
import os 
import os.path as osp

os.chdir("/home/DISCOVER_summer2022/xusc/exp/butd_detr")
os.getcwd()
import numpy as np 
import sys
from src.arkitscenes_dataset import ARKitSceneDataset
from loguru import logger
from my_script.pc_utils import *
from my_script.utils import *



def save_(data,scene_name,save_root,has_color= True,flag = "debug"):
      #* for teacher or student 
      
    make_dirs(osp.join(save_root,scene_name))

    if has_color:
        write_pc_as_ply(
                    data['point_clouds'],
                    os.path.join(save_root,scene_name, '%s_gt_%s.ply'%(scene_name,flag))
                )
    else :
        write_ply(
            data['point_clouds'],
            os.path.join(save_root,scene_name, '%s_gt_%s.ply'%(scene_name,flag))
        )

    #* open3d  format bounding box 
    np.savetxt(os.path.join(save_root, scene_name,'%s_box_%s.txt'%(scene_name,flag)),
            data['all_bboxes'][data['all_bbox_label_mask']],fmt='%s')

    

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## 2.  检查ARKitScene datasets  getitem 的数据格式

In [2]:
point_num = 50000
arkit = ARKitSceneDataset('train', num_points=point_num,augment=True,data_root='datasets/arkitscenes')
demo = arkit.__getitem__(1)
demo_attr  = get_attr(demo)
print_attr_shape(demo)


2022-10-22 00:30:44.201 | INFO     | my_script.utils:print_attr_shape:313 - box_label_mask : (132,)
2022-10-22 00:30:44.202 | INFO     | my_script.utils:print_attr_shape:313 - center_label : (132, 3)
2022-10-22 00:30:44.203 | INFO     | my_script.utils:print_attr_shape:313 - sem_cls_label : (132,)
2022-10-22 00:30:44.204 | INFO     | my_script.utils:print_attr_shape:313 - size_gts : (132, 3)
2022-10-22 00:30:44.204 | INFO     | my_script.utils:print_attr_shape:295 - scan_ids : 40753686
2022-10-22 00:30:44.205 | INFO     | my_script.utils:print_attr_shape:313 - point_clouds : (50000, 6)
2022-10-22 00:30:44.205 | INFO     | my_script.utils:print_attr_shape:295 - utterances : mug . table . guitar . toilet brush . night light . storage container . stand . rug . scanner . shampoo . soap . chair . not mentioned
2022-10-22 00:30:44.205 | INFO     | my_script.utils:print_attr_shape:313 - tokens_positive : (132, 2)
2022-10-22 00:30:44.206 | INFO     | my_script.utils:print_attr_shape:313 - posi

# save for vis test 

In [4]:

# debug_path = "logs/debug"
# make_dirs(debug_path)
# for  idx, data in enumerate(arkit):
#     save_(data,data['scan_ids'])
#     logger.info(f"data['scan_ids']:{data['scan_ids']}")
#     if idx ==5 :
#         break


2022-10-21 10:57:13.813 | INFO     | src.arkitscenes_dataset_uncalibrated:__getitem__:185 - before delete ceil :(104513, 6)
2022-10-21 10:57:13.821 | INFO     | src.arkitscenes_dataset_uncalibrated:__getitem__:192 - after  delete ceil :(83610, 6)
2022-10-21 10:57:18.304 | INFO     | __main__:<module>:8 - data['scan_ids']:40753679
2022-10-21 10:57:18.309 | INFO     | src.arkitscenes_dataset_uncalibrated:__getitem__:185 - before delete ceil :(97751, 6)
2022-10-21 10:57:18.315 | INFO     | src.arkitscenes_dataset_uncalibrated:__getitem__:192 - after  delete ceil :(78200, 6)
2022-10-21 10:57:22.795 | INFO     | __main__:<module>:8 - data['scan_ids']:40753686
2022-10-21 10:57:22.820 | INFO     | src.arkitscenes_dataset_uncalibrated:__getitem__:185 - before delete ceil :(193440, 6)
2022-10-21 10:57:22.831 | INFO     | src.arkitscenes_dataset_uncalibrated:__getitem__:192 - after  delete ceil :(154752, 6)
2022-10-21 10:57:27.185 | INFO     | __main__:<module>:8 - data['scan_ids']:40776203
2022

2022-10-20 14:35:27.498 | INFO     | __main__:<module>:4 - demo['scan_ids']:40753679
